In [32]:
from astropy.coordinates import SkyCoord

from astroquery.sdss import SDSS

from IPython import display

In [7]:
spec = get_spectra(plate=285, mjd=51930, fiberID=232)[0]

In [39]:
url = 'http://skyserver.sdss.org/dr12/SkyserverWS/ImgCutout/getjpeg?ra={}&dec={}&width=256&height=256&scale=.1'.format(spec[0].header['PLUG_RA'], spec[0].header['PLUG_DEC'])
display.Image(url=url)

... more coming ...